In [39]:
from explainers.gnninterpreter import * 


In [40]:
import torch
from torch import nn

import libraries.dataLoaderWrapper 

In [41]:
dataset = torch.load('model/loader_house_gnn_interpreter_model.pt')

In [42]:
model = GCNClassifier(node_features=3,
                      num_classes=2,
                      hidden_channels=32)

model.load_state_dict(torch.load('model/house_gnn_interpreter_model.pt'))

<All keys matched successfully>

In [43]:
mean_embeds = dataset.mean_embeddings(model)

In [44]:
dataset[0].x

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])

In [45]:
trainer = {}
sampler ={}

In [46]:
cls_idx = 0
trainer[cls_idx] = Trainer(
    sampler=(s := GraphSampler(
        max_nodes=20,
        num_node_cls=3,
        temperature=0.15,
        learn_node_feat=True
    )),
    discriminator=model,
    criterion=WeightedCriterion([
        dict(key="logits", criterion=ClassScoreCriterion(class_idx=cls_idx, mode='maximize'), weight=1),
        dict(key="embeds", criterion=EmbeddingCriterion(target_embedding=mean_embeds[cls_idx]), weight=10),
        dict(key="logits", criterion=MeanPenalty(), weight=0),
        dict(key="omega", criterion=NormPenalty(order=1), weight=1),
        dict(key="omega", criterion=NormPenalty(order=2), weight=1),
        dict(key="xi", criterion=NormPenalty(order=1), weight=0),
        dict(key="xi", criterion=NormPenalty(order=2), weight=0),
        # dict(key="eta", criterion=NormPenalty(order=1), weight=0),
        # dict(key="eta", criterion=NormPenalty(order=2), weight=0),
        dict(key="theta_pairs", criterion=KLDivergencePenalty(binary=True), weight=0),
    ]),
    optimizer=(o := torch.optim.SGD(s.parameters(), lr=1)),
    scheduler=torch.optim.lr_scheduler.ExponentialLR(o, gamma=1),
    dataset=dataset,
    budget_penalty=BudgetPenalty(budget=10, order=2, beta=1),
    target_probs={cls_idx: (0.9, 1)},
    k_samples=16
)

In [47]:
trainer[0].train(2000)

  0%|          | 0/2000 [00:00<?, ?it/s]

True

In [55]:
graph = trainer[0].evaluate(threshold=0.5, show=False)
labels = [graph.nodes[node]['label'] for node in graph.nodes]
import networkx as nx
nx.draw(graph, nx.kamada_kawai_layout(graph), node_color=labels)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (7x39 and 3x32)

In [52]:
graph

AttributeError: 'Graph' object has no attribute 'data'

In [53]:
node_data = graph.nodes(data=True)

# Extract labels and assign colors based on the color map
node_colors = [label_color_map[data['label']] for _, data in node_data]

In [54]:
node_data

NodeDataView({3: {'label': 2}, 8: {'label': 1}, 7: {'label': 2}, 10: {'label': 2}, 11: {'label': 0}, 15: {'label': 1}, 19: {'label': 2}})

In [ ]:
nx.draw(graph, with_labels=True, node_color=node_data.data['label'])

AttributeError: 'NodeDataView' object has no attribute 'data'